# COMP3132 - Lab Week 1

# Building an LLM-Powered Chatbot: A Hands-On Guide in Google Colab

## Google Colab Configuration

### Some of our labs this semester will be painfully slow if without a GPU. The easies way to get access to a GPU accelerated Jupyter notebook is to enable the `T4 GPU runtime` on Google Colab:

### 1. Navigate to `Runtime`.
### 2. Select `Change runtime type`.
### 3. Choose `Hardware accelerator`.
### 4. Select `T4 GPU`.

### **Note:** This notebook can be run on `CPU` without any noticeable difference in performance.

In [ ]:
from IPython.display import Image, display

In [ ]:
!pip install python-dotenv
!pip install jupyter_bokeh

# Online Chatbot

### Go to https://api.together.ai/playground/meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo to chat with the model online on `togerther.ai` website and play with the chatbot by changing the configurations and hyper-parameters

# A Brief Theory




## Training a Language Model

In [ ]:
image_path = 'https://raw.githubusercontent.com/PyDataGBC/PyML2026/refs/heads/main/assets/LLM_train.png'
display(Image(image_path, width=600))

## Base Vs. Chat Models

### After training the LLMs with this paradigm on a very large amount of data (such as the entire internet), we will have a model, also known as a `foundation` model or `base` model, that can predict the next word repeatedly to form a sentence.

### To enable the model to engage in conversations, we further fine-tune the base model using instructions, such as question-answer pairs. These models are referred to as `instruction-tuned` or `chat` models.

### You can observe the different behaviors of the base and instruction-tuned models in the following slide.

In [ ]:
image_path = 'https://raw.githubusercontent.com/PyDataGBC/PyML2026/refs/heads/main/assets/baseVSinstruct.png'
display(Image(image_path, width=800))


## Interacting with Model Programmatically

In [ ]:
image_path = 'https://raw.githubusercontent.com/PyDataGBC/PyML2026/refs/heads/main/assets/modelaccess.png'
display(Image(image_path, width=500))

# Designing Our Own Chatbot

## API Call to the Model

### Getting API KEY

#### - Go to https://api.together.xyz/settings/api-keys to get your API key.

#### Importing the API Key to Colab

1. On the left-side vertical menu, select the `key` icon.
2. Add a secret key with the following details:
   - **Name**: `TOGETHER_API_KEY`
   - **Value**: `<your API key>`

In [ ]:
# from google.colab import userdata
# api_key = userdata.get('TOGETHER API KEY')
api_key = 'tgp_v1_GKBdA9ptqAemxc-i2VHL1aYLG6g9ikaovTSUrxCT_A'


### Function to call the API

In [ ]:
import os
# from dotenv import load_dotenv, find_dotenv
import warnings
import requests
import json
import time

warnings.filterwarnings('ignore')
url = "https://api.together.xyz/inference"

headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }


import time
def llama(prompt,
          add_inst=True,
          model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
          temperature=0.0,
          max_tokens=1024,
          verbose=False,
          url=url,
          headers=headers,
          base = 2, # number of seconds to wait
          max_tries=3):

    if add_inst:
        prompt = f"[INST]{prompt}[/INST]"

    if verbose:
        print(f"Prompt:\n{prompt}\n")
        print(f"model: {model}")

    data = {
            "model": model,
            "prompt": prompt,
            "temperature": temperature,
            "max_tokens": max_tokens
        }

    # Allow multiple attempts to call the API incase of downtime.
    # Return provided response to user after 3 failed attempts.
    wait_seconds = [base**i for i in range(max_tries)]

    for num_tries in range(max_tries):
        try:
            response = requests.post(url, headers=headers, json=data)
            return response.json()['output']['choices'][0]['text']
        except Exception as e:
            if response.status_code != 500:
                return response.json()

            print(f"error message: {e}")
            print(f"response object: {response}")
            print(f"num_tries {num_tries}")
            print(f"Waiting {wait_seconds[num_tries]} seconds before automatically trying again.")
            time.sleep(wait_seconds[num_tries])

    print(f"Tried {max_tries} times to make API call to get a valid response object")
    print("Returning provided response")
    return response


### **Note:** Default model is `"meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo"` but can you can change it by finding the model name from https://api.together.ai/playground/chat

## General testing the model

In [ ]:
print("prompt 1")

In [ ]:
# pass prompt to the llama function, store output as 'response' then print
prompt = "Tell me a funny joke about software developers."
response = llama(prompt)  # temperature is a hyperparameter that controls randomness in the response
print(response)

In [ ]:
prompt = "What is the capital of France?"
mistral = 'mistralai/Ministral-3-14B-Instruct-2512'
response = llama(prompt, verbose=True, add_inst=False, model=mistral)  # verbose=True will print the prompt
print(response)


In [ ]:
print(response['choices'][0]['text'])


## Exercise 1: General testing

#### 1. Change the `temprarature` parameter from 0.0 to 0.9 and see the difference in the responses.
#### Note: temperature parameter is a number between 0.0 and 1.0. It controls the randomness of the responses.

In [ ]:
#your code here
print(llama("What tags are used as controls for the chat mode?", add_inst=True))


## Role prompting

#### - Roles give context to LLMs what type of answers are desired.
#### - LLMs often gives more consistent responses when provided with a role.
#### - First, try standard prompt and see the response.

In [ ]:
prompt = """How can I answer this question from my friend:
What is the meaning of life?"""

response = llama(prompt)
print(response)

###  Now, try it by giving the model a `role`, and within the role, a `tone` using which it should respond with.

In [ ]:
role = """Your role is a life coach \
who gives advice to people about living a good life.\
You attempt to provide unbiased advice.
You respond in the tone of an English pirate.
"""

prompt = f"""
{role}
How can I answer this question from my friend:
What is the meaning of life?
"""

#print(prompt)

response = llama(prompt)
print(response)


## Excercise 2: Role prompting

#### Role: Beginner python tutor
#### Task: Explain how to create a list and add an element to it.

In [ ]:
# your code here
prompt = "Your are a beginner python tutor. Explain how to create a list and add an element to it."
print(llama(prompt))


#### Change the role to `friendly coding mentor` and see how the response changes for the same task.

In [ ]:
# your code here
prompt = "Your are a friendly coding mentor. Explain how to create a list and add an element to it."
print(llama(prompt))


## Asking follow-up questions

### Does the model have memory of the previous conversation?

In [ ]:
prompt_1 = "What are fun activities I can do this weekend?"

response_1 = llama(prompt_1)
print(response_1)

In [ ]:
prompt_2 = "Which of these would be good for my health?"
response_2 = llama(prompt_2)
print(response_2)

#### Is the the second answer related to the first answer?
#### **Note:** LLMs are `stateless` models, so they don't have memory of the previous conversation.

## Multi-turn prompting (chatting)
#### In order to give the model memory of the previous conversation, you need to provide prior prompts and responses as part of the context of each new turn in the conversation.

In [ ]:
image_path = 'https://raw.githubusercontent.com/PyDataGBC/PyML2026/refs/heads/main/assets/multi_turn.png'
display(Image(image_path, width=600))

### Note: you don't need `end tag (</s>)` for the last prompt.

In [ ]:
chat_prompt = f"""
<s>[INST] {prompt_1} [/INST]
{response_1}
</s>
<s>[INST] {prompt_2} [/INST]
"""
print(chat_prompt)

### Note: pay attention to add_inst (add instruction) argument below

In [ ]:
response_2 = llama(chat_prompt,
                 add_inst=False)

In [ ]:
print(response_2)

### Helper function to handle multi-turn prompting

### **Note:** You don’t need to understand every part of the helper function. In the next section, you’ll see how to use it in your code.

In [ ]:
def llama_chat(prompts,
               responses,
               model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
               temperature=0.0,
               max_tokens=1024,
               verbose=False,
               url=url,
               headers=headers,
               base=2,
               max_tries=3
              ):

    prompt = get_prompt_chat(prompts,responses)

    # Allow multiple attempts to call the API incase of downtime.
    # Return provided response to user after 3 failed attempts.
    wait_seconds = [base**i for i in range(max_tries)]

    for num_tries in range(max_tries):
        try:
            response = llama(prompt=prompt,
                             add_inst=False,
                             model=model,
                             temperature=temperature,
                             max_tokens=max_tokens,
                             verbose=verbose,
                             url=url,
                             headers=headers
                            )
            return response
        except Exception as e:
            if response.status_code != 500:
                return response.json()

            print(f"error message: {e}")
            print(f"response object: {response}")
            print(f"num_tries {num_tries}")
            print(f"Waiting {wait_seconds[num_tries]} seconds before automatically trying again.")
            time.sleep(wait_seconds[num_tries])

    print(f"Tried {max_tries} times to make API call to get a valid response object")
    print("Returning provided response")
    return response


def get_prompt_chat(prompts, responses):
  prompt_chat = f"<s>[INST] {prompts[0]} [/INST]"
  for n, response in enumerate(responses):
    prompt = prompts[n + 1]
    prompt_chat += f"\n{response}\n </s><s>[INST] \n{ prompt }\n [/INST]"

  return prompt_chat

### How to use the helper function

In [ ]:
prompt_1 = "What are fun activities I can do this weekend?"
response_1 = llama(prompt_1)
print(response_1)


In [ ]:
print(response_1)

In [ ]:
prompt_2 = "Which of these would be good for my health?"

In [ ]:
prompts = [prompt_1,prompt_2]
responses = [response_1]

In [ ]:
# Pass prompts and responses to llama_chat function
response_2 = llama_chat(prompts,responses)

In [ ]:
print(response_2)

### Excercise 3: Multi-turn prompting

### Ask this follow-up question: "Which of these activites would be fun with friends?"

In [ ]:
prompt_3 = "Which of these activites would be fun with friends?"

#your code here

In [ ]:
prompts = [prompt_1,prompt_2, prompt_3]
responses = [response_1, response_2]

# Pass prompts and responses to llama_chat function
response_3 = llama_chat(prompts,responses)

print(response_3)

### OrderBot

#### We can `automate the collection of user prompts and model responses` to build a  OrderBot.

#### The OrderBot will take orders at a pizza restaurant.

In [ ]:
# Define the bot's role and menu
role = """
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then start collecting the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \

Primary Category: Pizza
  Secondary Category: \
    pepperoni pizza  12.95, 10.00, 7.00 \
    cheese pizza   10.95, 9.25, 6.50 \
    eggplant pizza   11.95, 9.75, 6.75 \
Primary Category: Sides
  Secondary Category: \
    fries 4.50, 3.50 \
    greek salad 7.25 \
Primary Category: Toppings: \
  Secondary Category: \
    extra cheese 2.00, \
    mushrooms 1.50 \
    sausage 3.00 \
    canadian bacon 3.50 \
    AI sauce 1.50 \
    peppers 1.00 \
Primary Category: Drinks \
  Secondary Category: \
    coke 3.00, 2.00, 1.00 \
    sprite 3.00, 2.00, 1.00 \
    bottled water 5.00 \

the price based on size example:
pepperoni pizza  large = 12.95,
pepperoni pizza  medium = 10.00,
pepperoni pizza  small = 7.00 \

For all items also check the size with the customer first
Do not forget to ask for drinks and sides.
Do not add any items extra by yourself
"""
 # accumulate messages

## Excercise 4: Orderbot

In [ ]:
prompts = []
responses = []

prompts.append(role)
responses.append('Hi, what would you like to order today?')

print(prompts)
print(responses)


In [ ]:
print(responses[0])
while True:
    p = input("User: ")
    if p.lower() in ['done', 'exit', 'quit', 'bye']:
        print("Chatbot: Thank you for your order! Have a great day!")
        break

    prompts.append(p)
    response = llama_chat(prompts, responses)
    responses.append(response)
    print(f"\nChatbot: {response}\n")
    # terminate the loop if the user types 'done', 'exit', 'quit', or bye


### Printing the order

In [ ]:
role = 'create a json summary of the food order. Itemize the price for each item\
 The fields should be 1) pizza, include type of pizza and size and price 2) list of toppings with price 3) list of drinks, include size and price\
          4) list of sides include size and price 5)total price - just include items in my order and do not add anything by yourself'

messages = prompts.copy()
messages.append(role)

response = llama_chat(messages, responses)
print(response)